In [0]:
# Delta Sharing Setup for Rearc Reviewer
# This script creates a Delta Share with access to both raw BLS data and report tables

# Configuration
catalog = 'rearcquest'
rawSchema = 'raw'
reportsSchema = 'default'
shareName = 'rearc_bls_data_share'
recipientName = 'rearc_reviewer'
recipientEmail = 'reviewer@rearc.io'

In [0]:
# Create the Delta Share
try:
    spark.sql(f"""
        CREATE SHARE IF NOT EXISTS {shareName} 
        COMMENT 'BLS raw tables and reports for Rearc reviewer'
    """)
except Exception as e:
    if "already exists" not in str(e).lower():
        raise e

In [0]:
# Add all raw BLS tables to the share. This is known to be inefficient, but it's a one-time run.
rawTables = spark.sql(f"SHOW TABLES IN {catalog}.{rawSchema}").collect()

for table in rawTables:
    tableName = f"{catalog}.{rawSchema}.{table.tableName}"
    try:
        spark.sql(f"ALTER SHARE {shareName} ADD TABLE {tableName}")
    except Exception as e:
        if "already exists" in str(e).lower():
            pass
        else:
            print(f"Failed to add {tableName}: {str(e)}")

In [0]:
# Add report tables to the share.
reportTables = ['best_year','population_stats','bls_with_population']

for tableName in reportTables:
    fullTableName = f"{catalog}.{reportsSchema}.{tableName}"
    try:
        spark.sql(f"ALTER SHARE {shareName} ADD TABLE {fullTableName}")
        reportTablesAdded.append(fullTableName)
    except Exception as e:
        if "already exists" in str(e).lower():
            pass
        elif "not found" in str(e).lower():
            print(f"Table not found: {fullTableName}")
        else:
            print(f"Failed to add {fullTableName}: {str(e)}")

In [0]:
# Create recipient for Rearc reviewer
try:
    spark.sql(f"""
        CREATE RECIPIENT IF NOT EXISTS {recipientName}
        COMMENT 'Rearc reviewer - {recipientEmail}'
    """)
except Exception as e:
    if "already exists" not in str(e).lower():
        raise e

# Grant SELECT permission on the share to the recipient
try:
    spark.sql(f"GRANT SELECT ON SHARE {shareName} TO RECIPIENT {recipientName}")
except Exception as e:
    if "already has" not in str(e).lower():
        raise e

In [0]:
# Retrieve the activation link for the recipient
recipientInfo = spark.sql(f"DESCRIBE RECIPIENT {recipientName}").collect()

activationLink = None
for row in recipientInfo:
    if row.info_name == "activation_link":
        activationLink = row.info_value
        break

if activationLink:
    print(f"Recipient: {recipientName}")
    print(f"Email: {recipientEmail}")
    print(f"\nActivation Link:")
    print(activationLink)
else:
    print("Could not retrieve activation link")
    print(f"Run: DESCRIBE RECIPIENT {recipientName}")

In [0]:
# Verify the complete Delta Sharing setup
print(f"Share: {shareName}")
print(f"Recipient: {recipientName} ({recipientEmail})")

print(f"\nTables in share:")
sharedTables = spark.sql(f"SHOW ALL IN SHARE {shareName}").collect()
sharedTables.print()